In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1691346772328_0006,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('T_previous_application_20230805').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
file='previous_application_20230805.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ts_file_generation = file.split("_")[-1].replace('.csv','') + '00'
print(ts_file_generation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2023080500

In [8]:
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_ingestion = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/001_ingestion'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
output_lake = 't_previous_application_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/001_ingestion/previous_application_20230805.csv

In [11]:
pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)
pouso.registerTempTable("pouso")
pouso.cache()
qtd=pouso.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 1670214

In [12]:
for col in pouso.columns:
    print(col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SK_ID_PREV,
SK_ID_CURR,
NAME_CONTRACT_TYPE,
AMT_ANNUITY,
AMT_APPLICATION,
AMT_CREDIT,
AMT_DOWN_PAYMENT,
AMT_GOODS_PRICE,
WEEKDAY_APPR_PROCESS_START,
HOUR_APPR_PROCESS_START,
FLAG_LAST_APPL_PER_CONTRACT,
NFLAG_LAST_APPL_IN_DAY,
RATE_DOWN_PAYMENT,
RATE_INTEREST_PRIMARY,
RATE_INTEREST_PRIVILEGED,
NAME_CASH_LOAN_PURPOSE,
NAME_CONTRACT_STATUS,
DAYS_DECISION,
NAME_PAYMENT_TYPE,
CODE_REJECT_REASON,
NAME_TYPE_SUITE,
NAME_CLIENT_TYPE,
NAME_GOODS_CATEGORY,
NAME_PORTFOLIO,
NAME_PRODUCT_TYPE,
CHANNEL_TYPE,
SELLERPLACE_AREA,
NAME_SELLER_INDUSTRY,
CNT_PAYMENT,
NAME_YIELD_GROUP,
PRODUCT_COMBINATION,
DAYS_FIRST_DRAWING,
DAYS_FIRST_DUE,
DAYS_LAST_DUE_1ST_VERSION,
DAYS_LAST_DUE,
DAYS_TERMINATION,
NFLAG_INSURED_ON_APPROVAL,

In [13]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            int(date_format(current_date,'yyyyMM')) as ref,
            int(date_format(current_date,'yyyyMM')) as ref_partition,
            
            {tsfileger} as ts_file_generation,
            {tsfileger} as ts_file_generation_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            SK_ID_PREV,
            SK_ID_CURR,
            NAME_CONTRACT_TYPE,
            AMT_ANNUITY,
            AMT_APPLICATION,
            AMT_CREDIT,
            AMT_DOWN_PAYMENT,
            AMT_GOODS_PRICE,
            WEEKDAY_APPR_PROCESS_START,
            HOUR_APPR_PROCESS_START,
            FLAG_LAST_APPL_PER_CONTRACT,
            NFLAG_LAST_APPL_IN_DAY,
            RATE_DOWN_PAYMENT,
            RATE_INTEREST_PRIMARY,
            RATE_INTEREST_PRIVILEGED,
            NAME_CASH_LOAN_PURPOSE,
            NAME_CONTRACT_STATUS,
            DAYS_DECISION,
            NAME_PAYMENT_TYPE,
            CODE_REJECT_REASON,
            NAME_TYPE_SUITE,
            NAME_CLIENT_TYPE,
            NAME_GOODS_CATEGORY,
            NAME_PORTFOLIO,
            NAME_PRODUCT_TYPE,
            CHANNEL_TYPE,
            SELLERPLACE_AREA,
            NAME_SELLER_INDUSTRY,
            CNT_PAYMENT,
            NAME_YIELD_GROUP,
            PRODUCT_COMBINATION,
            DAYS_FIRST_DRAWING,
            DAYS_FIRST_DUE,
            DAYS_LAST_DUE_1ST_VERSION,
            DAYS_LAST_DUE,
            DAYS_TERMINATION,
            NFLAG_INSURED_ON_APPROVAL

        from
            pouso
    """.format(tsfileger=ts_file_generation,pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1670214

In [14]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_file_generation: integer (nullable = false)
 |-- ts_file_generation_partition: integer (nullable = false)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- SK_ID_PREV: string (nullable = true)
 |-- SK_ID_CURR: string (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- AMT_APPLICATION: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_DOWN_PAYMENT: string (nullable = true)
 |-- AMT_GOODS_PRICE: string (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: string (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: string (nullable = true)
 |-- RATE_DOWN_PAYMENT: string (nullable = true)
 |-- RATE_INTEREST_PRIMARY: string (nullable = true)
 |-- RATE_INTEREST_PRIVI

In [15]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+------------------+----------------------------+--------------+-----------------+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMEN

In [16]:
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
# |ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURPOSE|NAME_CONTRACT_STATUS|DAYS_DECISION|NAME_PAYMENT_TYPE    |CODE_REJECT_REASON|NAME_TYPE_SUITE|NAME_CLIENT_TYPE|NAME_GOODS_CATEGORY|NAME_PORTFOLIO|NAME_PRODUCT_TYPE|CHANNEL_TYPE           |SELLERPLACE_AREA|NAME_SELLER_INDUSTRY|CNT_PAYMENT|NAME_YIELD_GROUP|PRODUCT_COMBINATION     |DAYS_FIRST_DRAWING|DAYS_FIRST_DUE|DAYS_LAST_DUE_1ST_VERSION|DAYS_LAST_DUE|DAYS_TERMINATION|NFLAG_INSURED_ON_APPROVAL|
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
# |202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |2030495   |271877    |Consumer loans    |1730.43    |17145.0        |17145.0   |0.0             |17145.0        |SATURDAY                  |15                     |Y                          |1                     |0.0              |0.18283180324152784  |0.8673361522198731      |XAP                   |Approved            |-73          |Cash through the bank|XAP               |null           |Repeater        |Mobile             |POS           |XNA              |Country-wide           |35              |Connectivity        |12.0       |middle          |POS mobile with interest|365243.0          |-42.0         |300.0                    |-42.0        |-37.0           |0.0                      |
# |202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |2802425   |108129    |Cash loans        |25188.615  |607500.0       |679671.0  |null            |607500.0       |THURSDAY                  |11                     |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-164         |XNA                  |XAP               |Unaccompanied  |Repeater        |XNA                |Cash          |x-sell           |Contact center         |-1              |XNA                 |36.0       |low_action      |Cash X-Sell: low        |365243.0          |-134.0        |916.0                    |365243.0     |365243.0        |1.0                      |
# |202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |2523466   |122040    |Cash loans        |15060.735  |112500.0       |136444.5  |null            |112500.0       |TUESDAY                   |11                     |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-301         |Cash through the bank|XAP               |Spouse, partner|Repeater        |XNA                |Cash          |x-sell           |Credit and cash offices|-1              |XNA                 |12.0       |high            |Cash X-Sell: high       |365243.0          |-271.0        |59.0                     |365243.0     |365243.0        |1.0                      |
# |202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |2819243   |176158    |Cash loans        |47041.335  |450000.0       |470790.0  |null            |450000.0       |MONDAY                    |7                      |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-512         |Cash through the bank|XAP               |null           |Repeater        |XNA                |Cash          |x-sell           |Credit and cash offices|-1              |XNA                 |12.0       |middle          |Cash X-Sell: middle     |365243.0          |-482.0        |-152.0                   |-182.0       |-177.0          |1.0                      |
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
# only showing top 4 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)
lake.coalesce(1).write. \
partitionBy("ref_partition","ts_file_generation_partition","ts_proc_partition"). \
parquet(path_raw,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_previous_application_20230805

In [18]:
varDataFile = output_lake

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_file_generation,
                ts_file_generation_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6,7,8
            order by
                1,2,3,4,5,6,7,8
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|name_file                      |name_file_partition            |ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|t_previous_application_20230805|t_previous_application_20230805|202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |1670214      |
+-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+

In [23]:
# +-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |name_file                      |name_file_partition            |ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |t_previous_application_20230805|t_previous_application_20230805|202308|202308       |2023080500        |2023080500                  |20230806160022|20230806160022   |1670214      |
# +-------------------------------+-------------------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
path_control = os.path.join('s3://',bucket_control,'tb_0001_controle_procesamento_raw')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0001_controle_procesamento_raw